In [34]:
import pandas as pd
# df = pd.read_csv("foodsur.csv")
# df.head()
def renamer(df):
    for s in df:
        sorg = s
        s = s.upper()
        s = s.strip()
        if (s.find(' ') > 0):
            s = s.replace(" ", '_', 2)
            if (s.find(' ') > 0):
                s = s[:(s.find(' '))]
        if (s.find('/') > 0):
            s = s[:(s.find('/'))]
        df.rename(columns={sorg: s}, inplace=True)

def like_dislike(df):
    dataDict = list(df.T.to_dict().values())
    newDataDict = []
    for row in dataDict:
        count = 0
        if type(row["DISHES_YOU_LIKED"])==str:
            for tl in row["DISHES_YOU_LIKED"].split(","):            
                row2 = row.copy()
                row2["DISHES_NAME"] = tl.strip()
                row2["LIKE"] = 1
    #             print(row2)
                count = count+1
                newDataDict.append(row2)
        if type(row["DISHES_YOU_DISLIKED"])==str:
            for tl in row["DISHES_YOU_DISLIKED"].split(","):
                row2 = row.copy()
                row2["DISHES_NAME"] = tl.strip()
                row2["LIKE"] = -1
                count = count+1
                newDataDict.append(row2)
        if count==0:
           row2 = row.copy()
           row2["DISHES_NAME"] = ""
           row2["LIKE"] = 0
           newDataDict.append(row2)                    
    df2 = pd.DataFrame(newDataDict)
    del df2["DISHES_YOU_LIKED"]
    del df2["DISHES_YOU_DISLIKED"]
    df2.to_csv("food_sur_out.csv")
    return df2
    
df = pd.read_csv("foodsur.csv")
renamer(df)
del df["TIMESTAMP"]
df = like_dislike(df)
print(df.head(20))

         DISHES_NAME FOOD_TYPE  LIKE MOST_LIKED_CUISINE          YOUR_NAME  \
0    Chicken Biryani   Non Veg     1             Indian      Sanchit Mahto   
1   Tandoori Chicken   Non Veg     1             Indian      Sanchit Mahto   
2      Chicken Tikka   Non Veg     1             Indian      Sanchit Mahto   
3         Seek Kabob   Non Veg     1             Indian      Sanchit Mahto   
4              Pizza   Non Veg     1             Indian      Sanchit Mahto   
5                KFC   Non Veg     1             Indian      Sanchit Mahto   
6              Pasta   Non Veg    -1             Indian      Sanchit Mahto   
7              Dhosa   Non Veg    -1             Indian      Sanchit Mahto   
8               Idli   Non Veg    -1             Indian      Sanchit Mahto   
9       Sambhar Bada   Non Veg    -1             Indian      Sanchit Mahto   
10      Paneer Pulau   Non Veg    -1             Indian      Sanchit Mahto   
11   Chicken Biryani   Non Veg     1             Indian         

In [72]:
df= pd.read_csv("food_sur_out.csv")
df.sample(10)

,Sno,DISHES_NAME,FOOD_TYPE,LIKE,MOST_LIKED_CUISINE,YOUR_NAME,YOUR_WEIGHT
268,268,Pasta,Non Veg,1,Chinese,RAJESH VIDYARTHI,55
65,65,Chicken Biryani,Veg,-1,Indian,Shashwat Mishra,60
121,121,Chicken Tikka,Non Veg,1,Indian,Neha Rao,50
171,171,Mutton Biryani,Non Veg,-1,Indian,Kavita Reddy,51
111,111,Seek Kabob,Non Veg,1,Mughlai,Afroz Loya,84
124,124,Pizza,Non Veg,1,Indian,Neha Rao,50
18,18,Paneer Tikka,Non Veg,-1,Indian,Moin,75
5,5,KFC,Non Veg,1,Indian,Sanchit Mahto,70
79,79,KFC,Non Veg,1,Indian,Vaibhav Agrawal,80
190,190,Mix Veg,Veg,1,Indian,Arti Rathore,60


In [46]:
def get_top_rated(df, thr=80, no=10):
    df3 = df.groupby("DISHES_NAME").LIKE.mean()
    df4 = pd.DataFrame(df3)
    df4["no"] = df.groupby("DISHES_NAME").LIKE.count()
    df5 = df4[df4["no"]>thr]
    return df5.sort_values(["LIKE"], ascending=[0]).head(no)
get_top_rated(df, 8, 10)

,LIKE,no
DISHES_NAME,,
Dhosa,0.833333,24
KFC,0.600000,15
Chicken Biryani,0.565217,23
Pasta,0.500000,24
Pizza,0.478261,23
Seek Kabob,0.466667,15
Chicken Tikka,0.444444,18
Tandoori Chicken,0.428571,21
Idli,0.238095,21


In [51]:
def get_most_reviewed_popu(df, no=10):
    df3 = df.groupby("DISHES_NAME").LIKE.sum()
    df6 = pd.DataFrame(df3)
    df6["no"] = df.groupby("DISHES_NAME").LIKE.count()
    maxno = df6["no"].max()
    df6["LIKE"] = df6["LIKE"]/maxno
    return df6.sort_values(["LIKE"], ascending=[0]).head(no)
get_most_reviewed_popu(df)

,LIKE,no
DISHES_NAME,,
Dhosa,0.833333,24
Chicken Biryani,0.541667,23
Pasta,0.500000,24
Pizza,0.458333,23
KFC,0.375000,15
Tandoori Chicken,0.375000,21
Chicken Tikka,0.333333,18
Seek Kabob,0.291667,15
Idli,0.208333,21


In [52]:
def get_curated(df, thr=8, no=10):
    df3 = df.groupby("DISHES_NAME").LIKE.mean()
    df7 = pd.DataFrame(df3)
    df7["no"] = df.groupby("DISHES_NAME").LIKE.count()
    df8 = df7[df7["no"]>thr]
    maxno = df8["no"].max()
    df8["LIKE"] += df8["no"]/maxno - 0.5
    return df8.sort_values(["LIKE"], ascending=[0]).head(no)
get_curated(df)

F:\Anaconda 9\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,LIKE,no
DISHES_NAME,,
Dhosa,1.333333,24
Chicken Biryani,1.023551,23
Pasta,1.000000,24
Pizza,0.936594,23
Tandoori Chicken,0.803571,21
KFC,0.725000,15
Chicken Tikka,0.694444,18
Idli,0.613095,21
Paneer Tikka,0.613095,21


In [83]:
food_matrix = df.pivot_table(index='Sno', columns='DISHES_NAME', values='LIKE')
food_matrix.sample(5)

DISHES_NAME,ALL VEGAN FOOD,All Veg Items,Chicken Biryani,Chicken Kolhapuri,Chicken Tikka,Dhosa,Idli,KFC,Karele ki sabji,Mix Veg,Mutton Biryani,Paneer Pulau,Paneer Tikka,Pasta,Pizza,Sambhar Bada,Seek Kabob,Tandoori Chicken,all other chicken dishes,all other paneer dishes
Sno,,,,,,,,,,,,,,,,,,,,
133,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [89]:
def get_sim(food_matrix, food_name, thr=0, no=10):
    user_rating = food_matrix[food_name]
    similar_to_food=food_matrix.corrwith(user_rating)
    similar_to_food.sample(10)
    corr_SW = pd.DataFrame(similar_to_food, columns=['correlation'])
    df3 = df.groupby("DISHES_NAME").LIKE.sum()
    df6 = pd.DataFrame(df3)
    df6["no"] = df.groupby("DISHES_NAME").LIKE.count()
    corr_SW.dropna(inplace=True)
    corr_SW.head()
    corr_SW = corr_SW.join(df6['no'])
    return corr_SW[corr_SW['no'] > thr].sort_values(by='correlation', ascending=False).head(no)
get_sim(food_matrix, 'Dhosa')

,correlation,no
DISHES_NAME,,
Dhosa,1.0,24


In [75]:
def get_sim_users_reviews(df, food_matrix, uid):
    food_matrix.head()
    user_rating = food_matrix.loc[uid]
    similar_to_food=food_matrix.corrwith(user_rating, axis=1)
    similar_to_food.head()
    corr_SW = pd.DataFrame(similar_to_food, columns=['correlation'])
    corr_SW.dropna(inplace=True)
    corr_SW.head()

    coUsers = df.groupby("Sno").LIKE.count()
    coUsers = pd.DataFrame(coUsers)
    coUsers.columns = ["no"]
    coUsers.head()
    # dfNew.describe()
    corr_SW = corr_SW.join(coUsers['no'])
    corr_SW.head()
    corr_SW = corr_SW[corr_SW['no'] > 20].sort_values(by='correlation', ascending=False)
    corr_SW = corr_SW[corr_SW['correlation'] > 0.5].sort_values(by='correlation', ascending=False)
    corr_SW.head(10)
    corr_SW.shape
    corr_SW.head()
    uids = corr_SW.index
    newDf = df[df.Sno.isin(uids)]
    newDf.head(10)
    newDf.shape
    return newDf
    # dfNew

def get_sim_user_top_rated(df, food_matrix, uid):
    newDf = get_sim_users_reviews(df, food_matrix, uid)
    return get_top_rated(newDf, 10, 10)

In [76]:
get_sim_user_top_rated(df, food_matrix, 1)

F:\Anaconda 9\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
F:\Anaconda 9\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,LIKE,no
DISHES_NAME,,


In [80]:
get_sim_user_top_rated(df, movie_matrix, 2)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,rating,no
title,,
"Close Shave, A (1995)",4.782609,23
"Godfather, The (1972)",4.632653,98
Star Wars (1977),4.619048,126
Wings of Desire (1987),4.583333,12
12 Angry Men (1957),4.580645,31
Vertigo (1958),4.545455,44
Raging Bull (1980),4.514286,35
Sunset Blvd. (1950),4.500000,16
Fargo (1996),4.495798,119


In [117]:
def get_year_reviews(df, st_y, end_y):
    df['valid'] = df.title.apply(lambda x: (x.strip()[-5:-1]).isnumeric())
    newDf=df[df['valid']]
    newDf['year'] = newDf.title.apply(lambda x: int((x.strip()[-5:-1]))) 
    newDf=newDf[newDf['year']>=st_y]
    newDf=newDf[newDf['year']<=end_y]
    return newDf
newdf2 = get_year_reviews(df, 1998, 1998)
newdf2.shape
# newdf2[newdf2["year"]==False]
# newdf2["year"].describe()
# print(type(newdf2["year"][0]))
newdf2.sample(10)
get_top_rated(newdf2, 10,10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,rating,no
title,,
Apt Pupil (1998),4.100000,160
Primary Colors (1998),3.923077,13
"Big Lebowski, The (1998)",3.833333,42
Dangerous Beauty (1998),3.769231,13
Dark City (1998),3.500000,16
"Wedding Singer, The (1998)",3.486111,72
"Man in the Iron Mask, The (1998)",3.416667,12
Desperate Measures (1998),3.319444,72
"Replacement Killers, The (1998)",3.307692,39
